In [1]:
%git clone https://github.com/beratcmn/cortec.git
%cd ./cortec/
%ls

Cloning into 'cortec'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 87 (delta 1), reused 12 (delta 1), pack-reused 75
Receiving objects: 100% (87/87), 57.30 MiB | 31.73 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/cortec
data  README.md  requirements.txt  src


In [2]:
%pip install -r requirements.txt

In [3]:
%pip -q install langchain openai tiktoken chromadb pypdf gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 80.8 MB/s 

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

In [5]:
import os
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["OPENAI_API_BASE"] = "https://demo-moh-opeanaij-ape.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "a275c1b36f42488eb96add2a955aba48"

In [6]:
model = AzureChatOpenAI(
    deployment_name="Demo-CORTEC-gpt-4",
    model_name="gpt-4",
    temperature=0
)

In [7]:
result = model(
    [
        HumanMessage(content="What are some touristic places in Paris?")
    ]
)

print(result.content)

1. Eiffel Tower: The most iconic symbol of Paris, offering panoramic views of the city.

2. Louvre Museum: The world's largest art museum and a historic monument in Paris.

3. Notre-Dame Cathedral: A masterpiece of French Gothic architecture, and one of the largest and most well-known church buildings in the world.

4. Montmartre: Known for its bohemian past and the beautiful Sacré-Cœur Basilica, there are also many unique shops and cafes to explore.

5. Champs-Élysées: One of the most famous streets in the world, leading up to the Arc de Triomphe.

6. Palace of Versailles: A royal château in Versailles, known for its Hall of Mirrors, the jewel-like Royal Opera, and the royal apartments.

7. Seine River: Offers boat tours that provide a unique way to see the city's famous landmarks.

8. Musée d'Orsay: An art museum housed in a former railway station, it holds mainly French art dating from 1848 to 1914.

9. Sainte-Chapelle: Known for its stunning stained glass windows, it's considered a

In [8]:
embeddings = OpenAIEmbeddings(
    deployment="Demo-CORTEC-text-embedding-ada",
    model="text-embedding-ada-002",
    openai_api_base="https://demo-moh-opeanaij-ape.openai.azure.com/",
    openai_api_type="azure",
    chunk_size=16,
)

In [9]:
!pwd

/content/cortec


In [10]:
loader = DirectoryLoader('./data/CORTEC_cleaned/', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [12]:
len(texts)

1540

In [13]:
persist_directory = 'db'

In [14]:
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)

In [15]:
#vectordb.persist()
#vectordb = None

In [16]:
#vectordb = Chroma(persist_directory=persist_directory,
#                  embedding_function=embeddings)

In [17]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [18]:
tool = create_retriever_tool(
    retriever,
    "search_cortec_documents",
    "Searches and returns documents regarding the CORTEC."
)
tools = [tool]

In [19]:
agent_executor = create_conversational_retrieval_agent(model, tools, verbose=True)

In [20]:
agent_executor.agent.prompt.messages

[SystemMessage(content='Do your best to answer the questions. Feel free to use any tools available to look up relevant information, only if necessary', additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history'),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='{input}', template_format='f-string', validate_template=True), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [21]:
template = """
You are an AI assistant representing the company CORTEC.

Do your best to answer the questions. Feel free to use any tools available to look up relevant information, only if necessary
""".strip()

agent_executor.agent.prompt.messages[0].content = template

In [22]:
result = agent_executor({"input": "hello"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


In [23]:
result = agent_executor({"input": "what is Compatibility & Interactions between Cathodic Protection and a Vapor?"})



> Entering new AgentExecutor chain...
Cathodic protection is a technique used to control the corrosion of a metal surface by making it the cathode of an electrochemical cell. This is achieved by placing in contact with the metal to be protected another more easily corroded metal to act as the anode of the electrochemical cell.

Vapor corrosion inhibitors (VCIs) are a type of corrosion inhibitor that are transported in a closed environment to the site of corrosion by volatilization from a source. In vapor phase, VCIs have an attraction to the charged surface of the metal, where they form a thin, monomolecular layer that protects against corrosive elements.

The compatibility and interactions between cathodic protection and a vapor (like VCI) can be complex. Both methods aim to prevent corrosion, but they work in different ways and can sometimes interfere with each other. For example, the presence of a VCI might affect the performance of the cathodic protection system. On the other han

In [24]:
import gradio as gr
import time


def get_message(userInput: str):
    return agent_executor({"input":userInput})["output"]


with gr.Blocks(css="footer {visibility: hidden}") as app:
    app.title = "Web-UI Demo"
    # gr.Markdown("""
    # This is a demo of a Smartenop Chatbot. Enter a question and the AI will respond.
    # Current model: **MoH Healthbot**
    # """)

    # with gr.Tab("Healthbot"):
    chatbot = gr.Chatbot(show_label=False).style(height=400)

    with gr.Row():
        with gr.Column(scale=0.90):
            msg = gr.Textbox(placeholder="Ask a question", show_label=False).style(container=False)
        with gr.Column(scale=0.05):
            submit = gr.Button("Send")
        with gr.Column(scale=0.05):
            clear = gr.Button("🗑️")

    userMessage = ""

    def user(user_message, history):
        global userMessage
        userMessage = user_message
        return "", history + [[user_message, None]]

    def bot(history):
        bot_message = get_message(userMessage)
        history[-1][1] = ""
        for character in bot_message:
            history[-1][1] += character
            time.sleep(0.05)
            yield history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

    submit.click(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

    clear.click(lambda: None, None, chatbot, queue=False)

    # with gr.Tab("Settings"):
    #    gr.Label("Settings")


if __name__ == "__main__":
    gr.close_all()
    app.queue()
    app.launch(server_port=8000, share=True)

<ipython-input-24-a766c35cdf9c>:17: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  chatbot = gr.Chatbot(show_label=False).style(height=400)
<ipython-input-24-a766c35cdf9c>:20: GradioDeprecationWarning: 'scale' value should be an integer. Using 0.9 will cause issues.
  with gr.Column(scale=0.90):
<ipython-input-24-a766c35cdf9c>:21: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  msg = gr.Textbox(placeholder="Ask a question", show_label=False).style(container=False)
<ipython-input-24-a766c35cdf9c>:22: GradioDeprecationWarning: 'scale' value should be an integer. Using 0.05 will cause issues.
  with gr.Column(scale=0.05):
<ipython-input-24-a766c35cdf9c>:24: GradioDeprecationWarning: 'scale' value should be an integer. Using 0.05 will cause issues.
  with gr.Column(scale=0.05):


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7f78fcecef0c81e60d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import time

while True:
    print("Still running...")
    time.sleep(600)

Still running...
Still running...
Still running...


> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `search_cortec_documents` with `Out-of-Service and Closure Requirements for Shop Fabricated Storage Tanks`


[Document(page_content='taken out-of-service, it shall be permanently closed. \n4. A storage tank system where financial responsibility is not maintained and demonstrated, pursuant to Rule 62-762.421, \nF.A.C., within 90 days of termination of the financial mechanism. \n(b) Closure of storage tank systems shall be performed by: \n1. Conducting a Closure Integrity Evaluation for shop fabricated storage tank systems as defined in subsection 62-762.201(8), \nF.A.C., and completing the Closure Integrity Evaluation Report Form for ASTs 62-762.901(7) (Closure Integrity Report),', metadata={'page': 37, 'source': 'data/CORTEC_cleaned/3.pdf'}), Document(page_content='(e) Storage tank systems wit